In [3]:
import os, tarfile
import pyBigWig
import pybedtools
import zipfile, gzip
#from gtfparse import read_gtf
import pandas as pd
pd.set_option('display.max_columns', None)

In [4]:
reference_genome_path = "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/genome_by_sequence/"
reference_genome_name = "chr1.fa"
data_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/gencode_annotation/"
file_name="gencodeV38.bb"
gencode_path= "/data/private/pdutta/Collab_data/Jim_work/mouse_gencode_annotation/"
gtf_annotation_filename= "gencode.vM33.annotation.gtf"
gff_annotation_filename= "gencode.v38.annotation.gff3"
file_path= data_path+file_name
gencode_gtf_file_path= gencode_path+ gtf_annotation_filename
gencode_gff_file_path= gencode_path+ gff_annotation_filename
reference_genome_file_path = reference_genome_path + reference_genome_name
data_save_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/region_wise_sequence"

In [5]:
gencode_gtf_file_path

'/data/private/pdutta/Collab_data/Jim_work/mouse_gencode_annotation/gencode.vM33.annotation.gtf'

In [34]:
column_names = [
    "seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"
]

df_gtf= pd.read_csv(gencode_gtf_file_path, sep="\t", header=None, comment='#', names=column_names)
df_gtf

,seqname,source,feature,start,end,score,strand,frame,attribute
0,chr1,HAVANA,gene,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; gene_type ""TEC..."
1,chr1,HAVANA,transcript,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
2,chr1,HAVANA,exon,3143476,3144545,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
3,chr1,ENSEMBL,gene,3172239,3172348,.,+,.,"gene_id ""ENSMUSG00000064842.3""; gene_type ""snR..."
4,chr1,ENSEMBL,transcript,3172239,3172348,.,+,.,"gene_id ""ENSMUSG00000064842.3""; transcript_id ..."
...,...,...,...,...,...,...,...,...,...
1032379,chr9,HAVANA,CDS,108691464,108691550,.,+,0,"gene_id ""ENSMUSG00000032598.9""; transcript_id ..."
1032380,chr9,HAVANA,exon,108691629,108691767,.,+,.,"gene_id ""ENSMUSG00000032598.9""; transcript_id ..."
1032381,chr9,HAVANA,CDS,108691629,108691767,.,+,0,"gene_id ""ENSMUSG00000032598.9""; transcript_id ..."
1032382,chr9,HAVANA,exon,108691866,108691946,.,+,.,"gene_id ""ENSMUSG00000032598.9""; transcript_id ..."


In [35]:
df_gtf['attribute'][0]

'gene_id "ENSMUSG00000102693.2"; gene_type "TEC"; gene_name "4933401J01Rik"; level 2; mgi_id "MGI:1918292"; havana_gene "OTTMUSG00000049935.1";'

In [39]:
row_counter = [0]
# Function to parse attributes into a dictionary
def parse_attributes(attributes_str):
    attributes = attributes_str.split("; ")
    attributes_dict = {}
    for attribute in attributes:
        if attribute:
            key, value = attribute.split(' ')
            attributes_dict[key] = value.strip('";')
    # Increment the counter
    row_counter[0] += 1
    
    # Print a message every 10000 rows
    if row_counter[0] % 50000 == 0:
        print(f"Processed {row_counter[0]} rows")
    return attributes_dict

In [40]:
# Apply the function to the attribute column and create a new dataframe
attributes_data = df_gtf['attribute'].apply(parse_attributes).apply(pd.Series)

Processed 50000 rows
Processed 100000 rows
Processed 150000 rows
Processed 200000 rows
Processed 250000 rows
Processed 300000 rows
Processed 350000 rows
Processed 400000 rows
Processed 450000 rows
Processed 500000 rows
Processed 550000 rows
Processed 600000 rows
Processed 650000 rows
Processed 700000 rows
Processed 750000 rows
Processed 800000 rows
Processed 850000 rows
Processed 900000 rows
Processed 950000 rows
Processed 1000000 rows


In [41]:
# Concatenate the new dataframe with the original dataframe
df_gtf = pd.concat([df_gtf, attributes_data], axis=1)
df_gtf.drop(columns='attribute', inplace=True)
df_gtf

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,mgi_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,protein_id,ccdsid,ont
0,chr1,HAVANA,gene,3143476,3144545,.,+,.,ENSMUSG00000102693.2,TEC,4933401J01Rik,2,MGI:1918292,OTTMUSG00000049935.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,3143476,3144545,.,+,.,ENSMUSG00000102693.2,TEC,4933401J01Rik,2,MGI:1918292,OTTMUSG00000049935.1,ENSMUST00000193812.2,TEC,4933401J01Rik-201,NA,Ensembl_canonical,OTTMUST00000127109.1,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,3143476,3144545,.,+,.,ENSMUSG00000102693.2,TEC,4933401J01Rik,2,MGI:1918292,OTTMUSG00000049935.1,ENSMUST00000193812.2,TEC,4933401J01Rik-201,NA,Ensembl_canonical,OTTMUST00000127109.1,1,ENSMUSE00001343744.2,NaN,NaN,NaN
3,chr1,ENSEMBL,gene,3172239,3172348,.,+,.,ENSMUSG00000064842.3,snRNA,Gm26206,3,MGI:5455983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,ENSEMBL,transcript,3172239,3172348,.,+,.,ENSMUSG00000064842.3,snRNA,Gm26206,3,MGI:5455983,NaN,ENSMUST00000082908.3,snRNA,Gm26206-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1032379,chr9,HAVANA,CDS,108691464,108691550,.,+,0,ENSMUSG00000032598.9,protein_coding,Nckipsd,2,MGI:1931834,OTTMUSG00000050675.2,ENSMUST00000035218.9,protein_coding,Nckipsd-201,1,CCDS,OTTMUST00000128467.1,7,ENSMUSE00000221318.4,ENSMUSP00000035218.8,CCDS23538.1,NaN
1032380,chr9,HAVANA,exon,108691629,108691767,.,+,.,ENSMUSG00000032598.9,protein_coding,Nckipsd,2,MGI:1931834,OTTMUSG00000050675.2,ENSMUST00000035218.9,protein_coding,Nckipsd-201,1,CCDS,OTTMUST00000128467.1,8,ENSMUSE00000221321.4,ENSMUSP00000035218.8,CCDS23538.1,NaN
1032381,chr9,HAVANA,CDS,108691629,108691767,.,+,0,ENSMUSG00000032598.9,protein_coding,Nckipsd,2,MGI:1931834,OTTMUSG00000050675.2,ENSMUST00000035218.9,protein_coding,Nckipsd-201,1,CCDS,OTTMUST00000128467.1,8,ENSMUSE00000221321.4,ENSMUSP00000035218.8,CCDS23538.1,NaN
1032382,chr9,HAVANA,exon,108691866,108691946,.,+,.,ENSMUSG00000032598.9,protein_coding,Nckipsd,2,MGI:1931834,OTTMUSG00000050675.2,ENSMUST00000035218.9,protein_coding,Nckipsd-201,1,CCDS,OTTMUST00000128467.1,9,ENSMUSE00000221323.4,ENSMUSP00000035218.8,CCDS23538.1,NaN


In [43]:
df_gtf.to_csv("/data/private/pdutta/Collab_data/Jim_work/mouse_gencode_annotation/gencode_gtf_annotation.tsv", sep="\t", index=False)